Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Deploy a model and monitor model data collectoin
_**Create and deploy a model directly from a notebook and monitor model data collection**_

---
---

## Contents
1. [Introduction](#Introduction)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
    1. Viewing run results
    1. Simple parameter sweep
    1. Viewing experiment results
    1. Select the best model
1. [Deploy](#Deploy)
    1. Register the model
    1. Create a scoring file
    1. Create the environment configuration (yml file for Conda and pip packages)
    1. Deploy the model as web service on Azure Kubernetes Service (AKS)
1. [Infere](#Infere)
    1. Test the Web Service witt a few prediction request to collect model data
    1. Inspect the Azure Blob Storage container to monitor the model data collection
    1. Clean up


---

## Introduction
Azure Machine Learning provides capabilities to control all aspects of model training and deployment directly from a notebook using the AML Python SDK. The main objectives of this notebook is to: 
* Train a regression model to predict if a patien would be diabetic based on a public diabetes dataset (https://aka.ms/diabetes-data).  
* Deploy that model as a web service in Azure Kubernetes Services, with ModelDataCollection enabled.
* Submit a few prediction requests to collect model data
* Monitor the model data collection in Azure Blobl Storage of the Machine Learning Workspace.

---

## Setup
Create an Azure Machine Learning servcie in Azure, and launch the studio. 

Create a Workspace and a Compute Instance to run this notebook.

For this notebook we need the Azure ML SDK and access to our workspace.  The following cell imports the SDK, checks the version, and accesses our already configured AzureML workspace. 

See more detail on [Git Integration](https://docs.microsoft.com/en-us/azure/machine-learning/concept-train-model-git-integration#:~:text=Azure%20Machine%20Learning%20provides%20a%20shared%20file%20system,work%20with%20Git%20via%20the%20Git%20CLI%20experience) if you need to upload this notebook in AML.

In [ ]:
import azureml.core
from azureml.core import Experiment, Workspace

# Check core SDK version number
print("This notebook was created using version 1.19.0 of the Azure ML SDK")
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")
print("")


ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

---

## Data
We will use the diabetes dataset for this experiement (https://aka.ms/diabetes-data). 

The dataset consists of 8 baseline variables for n=10000 diabetes patients: Pregnancies, PlasmaGlucose, DiastolicBloodPressure, TricepsThickness, SerumInsulin, BMI, DiabetesPedigree, and Age.

The dataset has one dichotomous outcome variable: Diebetic.

In [ ]:
from azureml.core import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split

# load the diabetes dataset from the same folder where this notebook is located
print("Loading Diabetes Data from the CSV file published in https://aka.ms/diabetes-data")
ds = pd.read_csv('https://aka.ms/diabetes-data')

# Separate features and labels
X, y = ds[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, ds['Diabetic'].values
# Split data into training set and test set (80% Training and 20% Testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

data = {
    "train":{"X": X_train, "y": y_train},        
    "test":{"X": X_test, "y": y_test}
}

print ("Data contains", len(data['train']['X']), "training samples and",len(data['test']['X']), "test samples")


---
## Train

Let's use scikit-learn to train a simple Ridge regression model.  We use AML to record interesting information about the model in an Experiment.  An Experiment contains a series of trials called Runs.  During this trial we use AML in the following way:
* We access an experiment from our AML workspace by name, which will be created if it doesn't exist
* We use `start_logging` to create a new run in this experiment
* We use `run.log()` to record a parameter, alpha, and an accuracy measure - the Mean Squared Error (MSE) to the run.  We will be able to review and compare these measures in the Azure Portal at a later time.
* We store the resulting model in the **working** directory, which is automatically captured by AML when the run is complete.
* We use `run.complete()` to indicate that the run is over and results can be captured and finalized

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import joblib

# Get an experiment object from Azure Machine Learning
experiment = Experiment(workspace=ws, name="train-diabetes-regression")

# Create a run object in the experiment
run =  experiment.start_logging()
# Log the algorithm parameter alpha to the run; where alpha is between 0 and 1
run.log('alpha', 0.03)

# Create, fit, and test the scikit-learn Ridge regression model
regression_model = Ridge(alpha=0.03)
regression_model.fit(data['train']['X'], data['train']['y'])
preds = regression_model.predict(data['test']['X'])

# Output the Mean Squared Error to the notebook and to the run
print('Mean Squared Error is', mean_squared_error(data['test']['y'], preds))
run.log('mse', mean_squared_error(data['test']['y'], preds))

# Save the model to the working directory 
model_file_name = 'diabetesregressionmodel.pkl'

joblib.dump(value = regression_model, filename = model_file_name)

# upload the model file explicitly into artifacts 
run.upload_file(name = model_file_name, path_or_stream = model_file_name)

# Complete the run
run.complete()

### Viewing run results
Azure Machine Learning stores all the details about the run in the Azure cloud.  Let's access those details by retrieving a link to the run using the default run output.  Clicking on the resulting link will take you to an interactive page presenting all run information.

In [ ]:
run

---
## Deploy
Now that we have trained the model, we want to deploy the model for real time inference.  The process of deploying a model involves
* registering a model in your workspace
* creating a scoring file containing init() and run() methods
* creating an environment settings file describing packages necessary for your scoring file
* creating a deployment configuration (for AKS in this example)
* deploying the model and packages as a web service in an AKS cluster

### Register a model
When using `run.register_model()` we supply a `model_name` that is meaningful for our scenario and the `model_path` of the model relative to the run.

In [ ]:
from azureml.core.model import Model
   
# Register the model with the workspace
model = Model.register(model_path = "diabetesregressionmodel.pkl",
                       model_name = "diabetesregressionmodel.pkl",
                       tags = {'area': "diabetes", 'type': "regression"},
                       description = "Ridge regression model to predict diabetes",
                       workspace =ws)

Once a model is registered, it is accessible from the list of models on the AML workspace.  If you register models with the same name multiple times, AML keeps a version history of those models for you.  The `Model.list()` lists all models in a workspace, and can be filtered by name, tags, or model properties.   

In [ ]:
# Find all models called "diabetesmodel" and display their version numbers
from azureml.core.model import Model
models = Model.list(ws, name="diabetesregressionmodel.pkl")
for m in models:
    print(m.name, m.version)

### Create a scoring file

Since your model file can essentially be anything you want it to be, you need to supply a scoring script that can load your model and then apply the model to new data. This script is your 'scoring file'. This scoring file is a python program containing, at a minimum, two methods init() and run(). The init() method is called once when your deployment is started so you can load your model and any other required objects. This method uses the get_model_path function to locate the registered model inside the docker container. The run() method is called interactively when the web service is called with one or more data samples to predict.

The schema decorators for pandas and numpy are required to implement the automatic swagger schema generation for input and output variables

The ModelDataCollector is included in the azureml-monitoring package. Make sure to install/upgrade to the latest package

After a successful run of the this script, the score.py file be created in the working folder


In [ ]:
# upgrade azureml sdk and monitoring
!pip install --upgrade azureml-sdk azureml-monitoring


In [ ]:
%%writefile score.py
import json
import pickle
import numpy as np
import pandas as pd
import joblib
from azureml.core.model import Model
from azureml.monitoring import ModelDataCollector # for model data collection

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType

def init():
    global model, inputs_datacollection, predictions_datacollection
    # collect inputs data send to, and predictions data received from, the model
    # The datacollection can be found under the storage account of the workspace
    # /modeldata/<subscriptionid>/<resourcegroup>/<workspace>/<webservice>/<model>/<version>/<designation>/<year>/<month>/<day>/data.csv
    inputs_datacollection = ModelDataCollector("diabetesregressionmodel.pkl", designation="inputs", feature_names=["Pregnancies", "PlasmaGlucose", "DiastolicBloodPressure", "TricepsThickness", "SerumInsulin", "BMI", "DiabetesPedigree", "Age"])
    predictions_datacollection = ModelDataCollector("diabetesregressionmodel.pkl", designation="predictions", feature_names=["Diebetic"])
    model_path = Model.get_model_path('diabetesregressionmodel.pkl')
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)

input_sample = pd.DataFrame(data=[{
            "Pregnancies": 0,
            "PlasmaGlucose": 171,
            "DiastolicBloodPressure": 80,
            "TricepsThickness": 34,
            "SerumInsulin": 23,
            "BMI": 43.51,
            "DiabetesPedigree": 1.21,
            "Age": 21,
            }])
output_sample = np.array([0])

@input_schema('data', PandasParameterType(input_sample))
@output_schema(NumpyParameterType(output_sample))

def run(data):
    try:
        result = model.predict(data)

        # save the inputs and prediction data collection from this run
        inputs_datacollection.collect(data) 
        predictions_datacollection.collect(result) 

        return result.tolist()
    except Exception as e:
        error = str(e)
        return error

### Create the environment settings 

The environment settings will also be exported into a yml file (myenv.yml) to verify the conda and pip packages.
The yml file will be in the working folder for this deployment

This step will create the python environment with the required conda and pip packages/dependencies. And then, it will create the inference configuration that will build the Docker container based on the scoring file and the environment configuration. The Docker image is transparent and will be created and registered behind the scenes with the AzureML SDK.

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

environment_name = 'deploytocloudenv'
env = Environment(environment_name)
env.python.conda_dependencies = CondaDependencies.create(conda_packages=['numpy','scikit-learn'],pip_packages=['azureml-defaults','azureml-monitoring','inference-schema[numpy-support]'])
inference_config = InferenceConfig(entry_script="score.py", environment=env)

with open ("myenv.yml","w") as f:
   f.write(env.python.conda_dependencies.serialize_to_string())

### Provision the AKS Cluster
This is a one time setup. You can reuse this cluster for multiple deployments after it has been created. If you delete the cluster or the resource group that contains it, then you would have to recreate it.

This step will take a few minutes.

In [ ]:
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your AKS cluster
aks_name = 'aks-cluster' 

# Verify that cluster does not exist already
try:
    aks_target = ComputeTarget(workspace=ws, name=aks_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # Use the default configuration (can also provide parameters to customize)
    prov_config = AksCompute.provisioning_configuration()

    # Create the cluster
    aks_target = ComputeTarget.create(workspace = ws, 
                                    name = aks_name, 
                                    provisioning_configuration = prov_config)

if aks_target.get_status() != "Succeeded":
    aks_target.wait_for_completion(show_output=True)

### Deploy the model as a web service to the AKS Cluster


In [ ]:
from azureml.core.webservice import AksWebservice

# Set the web service configuration (using default here)
# Model data collecoitn is not enabled by default. Enable here
aks_config = AksWebservice.deploy_configuration(collect_model_data=True)


In [ ]:
%%time
import datetime

aksservice_name ='aks-service'

print(str(datetime.datetime.now()))
# Create the webservice using all of the precreated configurations and our best model
aksservice = Model.deploy(workspace=ws,
                           name=aksservice_name,
                           models=[model],
                           inference_config=inference_config,
                           deployment_config=aks_config,
                           deployment_target=aks_target)

# Wait for the service deployment to complete while displaying log output
aksservice.wait_for_deployment(show_output = True)
print(aksservice.state)
print(aksservice.get_logs)

---
## Infere
The webservice has now been deployed. Next, we will submit a few prediction requests to the webservice in order to collect model data.

In [ ]:
import json

# Raw dataset (Actual Diabetic is 1)
test_sample = json.dumps({"data": [{
        "Pregnancies": 9,
        "PlasmaGlucose": 103,
        "DiastolicBloodPressure": 78,
        "TricepsThickness": 25,
        "SerumInsulin": 309,
        "BMI": 29.58,
        "DiabetesPedigree": 1.28,
        "Age": 43,}]})

test_sample = bytes(test_sample,encoding = 'utf8')
prediction = aksservice.run(input_data=test_sample)
print(prediction)


# Raw dataset (Actual Diabetic is 0)
test_sample = json.dumps({"data": [{
        "Pregnancies": 0,
        "PlasmaGlucose": 171,
        "DiastolicBloodPressure": 80,
        "TricepsThickness": 34,
        "SerumInsulin": 23,
        "BMI": 43.51,
        "DiabetesPedigree": 1.21,
        "Age": 21,}]})

test_sample = bytes(test_sample,encoding = 'utf8')
prediction = aksservice.run(input_data=test_sample)
print(prediction)


### Inspect the collected model data
Use the Azure Portal or the Storage Explorer tool to navigate to the storage account of the Azure Machine Learning Workspace

Follow the path under the container 'modeldata' as described in the [ModelDataCollector Class](https://docs.microsoft.com/en-us/python/api/azureml-monitoring/azureml.monitoring.modeldatacollector.modeldatacollector?view=azure-ml-py). 
You will notice there are two csv files in two different containers; inputs and predictions.
The csv files contain the input and prediction recrodsets from the run(s) above.




### Clean up

Delete the AKS web service to stop the compute and any associated billing.

In [ ]:
%%time
aksservice.delete()
model.delete()